In [13]:
import re

def remove_useless(input_str):
    input_str = input_str.replace("\n", " ")
    input_str = input_str.replace("\t", " ")
    remover = re.compile('<.*?>')
    input_str = re.sub(remover, '', input_str)
    input_str = input_str.strip()
    
    return input_str

In [20]:
def remove_betting(input_str):
    words = ["구.글", "구$글" "네온", "벳", "ㄴ ㅔ"]
    
    for word in words :
        if word in input_str:
            return False
        
    return True

In [27]:
from googleapiclient.discovery import build
import pandas as pd

comments = list()
videos = ('9IL39NPv96o', 'tKI5FO9iROg', 'EMTp4Gs_Q9o','EnGoVVj5sYY', 'DSdwo3ynX8w', 'RKoHeTgwdLA', '0TIcQuaIvFg', 'W79rP-EuKXE', 'sjohueRA_B4', 'lVTesJVmQM8', '9WUa_OpenxA')

for a_video in videos:
    video_id = a_video

    api_obj = build('youtube', 'v3', developerKey='google_api_key')
    response = api_obj.commentThreads().list(part='snippet,replies', videoId=video_id, maxResults=100).execute()

    while response:
        for item in response['items']:
            comment = item['snippet']['topLevelComment']['snippet']
            if remove_betting(comment['textDisplay']) == True and len(comment['textDisplay']) > 1:
                removed_comment = remove_useless(comment['textDisplay'])
                comments.append([removed_comment, comment['authorDisplayName'], comment['publishedAt'], comment['likeCount']])

                if item['snippet']['totalReplyCount'] > 0:
                    for reply_item in item['replies']['comments']:
                        reply = reply_item['snippet']
                        removed_reply = remove_useless(reply['textDisplay'])
                        comments.append([removed_reply, reply['authorDisplayName'], reply['publishedAt'], reply['likeCount']])

        if 'nextPageToken' in response:
            response = api_obj.commentThreads().list(part='snippet,replies', videoId=video_id, pageToken=response['nextPageToken'], maxResults=100).execute()
        else:
            break
 
df = pd.DataFrame(comments, columns=[['댓글','작성자','작성시간','좋아요수']])

In [28]:
df

,댓글,작성자,작성시간,좋아요수
0,야!,김재연,2022-01-25T11:00:19Z,0
1,세번째로 똑같은 말 할때 뿜음ㅋㅋ,이도윤,2022-01-21T13:07:16Z,0
2,사칭하지말라고 하는거에 46분을 쓰는 정성이 대단하십니다.,잡다한것,2022-01-20T02:08:06Z,0
3,욕하지마라 상수야,우주최강몰랑이,2022-01-17T11:45:38Z,0
4,중국 광고 같다,라임,2022-01-06T12:54:34Z,0
...,...,...,...,...
10315,이제 안속음 재미없으닌깐 하지마,주현성,2021-10-02T13:02:56Z,15
10316,형 신난거 보니꺼 너무 좋아❤️❤️❤️,o찬,2021-10-02T12:58:26Z,15
10317,형님 막곡은 에바였습니다......,더 진한 바나나우유,2021-10-02T12:58:21Z,5
10318,뮤ㅓ야 끔?,o찬,2021-10-02T12:58:15Z,0


In [29]:
df.to_csv('comments.csv', sep=',', encoding='utf-8-sig')